In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from ast import literal_eval

In [11]:
# Import dataset
df = pd.read_csv('../Data/4. Labeling/labeling.csv')

# Mengonversi kolom tokenisasi menjadi list
df['text_tokenize'] = df['text_tokenize'].apply(literal_eval)

# Menampilkan total dataset
print("Total Dataset :", len(df))

df.head()

Total Dataset : 4776


,video_id,comment_text,text_cleaned,text_removal,text_normalized,text_manual_replaced,text_tokenize,label
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,"[baik, bagaimana, turut, kalian]",0
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,"[betul, dok, betul, dok]",0
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa,betul dok mungkin efek sa,"[betul, dok, mungkin, efek, sa]",0
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu,bpom aneh kerja kok itu,"[bpom, aneh, kerja, kok, itu]",-1
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...,benar banget anak aku saja harus sirup minum o...,"[benar, banget, anak, aku, saja, harus, sirup,...",0


In [3]:
# Mengatur ulang indeks DataFrame setelah menghapus baris dengan nilai null
df.reset_index(drop=True, inplace=True)

# Menampilkan informasi singkat tentang DataFrame, seperti jumlah baris, tipe data, dan penggunaan memori
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4776 entries, 0 to 4775
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   video_id              4776 non-null   object
 1   comment_text          4776 non-null   object
 2   text_cleaned          4776 non-null   object
 3   text_removal          4776 non-null   object
 4   text_normalized       4776 non-null   object
 5   text_manual_replaced  4776 non-null   object
 6   text_tokenize         4776 non-null   object
 7   label                 4776 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 298.6+ KB


In [4]:
# Menyimpan label dalam variabel y dan fitur dalam variabel X
X = df.drop("label", axis=1)
y = df["label"]

In [7]:
# Menetapkan nilai SEED untuk memastikan pembagian data konsisten di setiap eksekusi
SEED = 42

prop_list = [(0.7, "70_30"), (0.8, "80_20"), (0.9, "90_10")]

for prop, name in prop_list:
    # No resampling (original)
    X_original, y_original = X, y
    
    # Menerapkan RandomUnderSampler
    rus = RandomUnderSampler(random_state=42)
    X_resampled_under, y_resampled_under = rus.fit_resample(X, y)

    # Menerapkan RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X_resampled_over, y_resampled_over = ros.fit_resample(X, y)

    for resample_type, X_resampled, y_resampled in [("original", X_original, y_original), ("under", X_resampled_under, y_resampled_under), ("over", X_resampled_over, y_resampled_over)]:
        # Memisahkan dataset menjadi data latih dan gabungan data validasi dan uji dengan proporsi yang ditentukan
        x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(X_resampled, y_resampled, test_size=1-prop, random_state=SEED, stratify=y_resampled)

        # Memisahkan gabungan data validasi dan uji menjadi data validasi dan uji dengan proporsi 50:50
        x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED, stratify=y_validation_and_test)

        # Menggabungkan fitur dan label untuk setiap bagian dataset
        train_data = pd.concat([x_train, y_train], axis=1)
        validation_data = pd.concat([x_validation, y_validation], axis=1)
        test_data = pd.concat([x_test, y_test], axis=1)

        # Menyimpan setiap bagian dataset ke file CSV
        train_data.to_csv(f'../Data/5. Split/{resample_type}/train_data_{name}_{resample_type}.csv', index=False)
        validation_data.to_csv(f'../Data/5. Split/{resample_type}/validation_data_{name}_{resample_type}.csv', index=False)
        test_data.to_csv(f'../Data/5. Split/{resample_type}/test_data_{name}_{resample_type}.csv', index=False)